In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import pygame
import sys

pygame 2.6.1 (SDL 2.28.4, Python 3.8.19)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Genrate Medications
meds_df = pd.DataFrame({
    'Medicine': [],
    'Lifespan': []
})
max_meds = 10
lifespan_mean = 25
lifespan_std = 10

LIFESPAN_MIN = 5

medicine = set(['{:04x}'.format(np.random.randint(0, 0xFFFF)) for _ in range(max_meds)])
# ensures there are max_meds unique values
while len(medicine) < max_meds: medicine.add('{:04x}'.format(np.random.randint(0, 0xFFFF)))
medicine = list(medicine)

lifespan = np.random.normal(lifespan_mean, lifespan_std, max_meds)
lifespan = np.round(np.maximum(lifespan, LIFESPAN_MIN))

meds_df['Medicine'] = medicine
meds_df['Lifespan'] = lifespan

meds_df.head()


Medicine  Lifespan
0     bbe0      13.0
1     38b5      21.0
2     eeed      21.0
3     3630      34.0
4     7f4f      36.0

In [26]:
# Generate Clients
client_count = 100
MAXLAT = 256
MAXLONG = 256
MAX_DOSAGE = 3

latitudes = np.random.randint(0, MAXLAT, client_count)
longitudes = np.random.randint(0, MAXLONG, client_count)

cs_df = pd.DataFrame({
    'Client': ['Client {:03d}'.format(i) for i in range(client_count)],
    'Lat': latitudes,
    'Long': longitudes,
    'Medicine': np.random.choice(medicine, client_count),
    'Dosage': np.random.randint(1, MAX_DOSAGE, client_count)
})
# cs_df.index.name = 'Client'
cs_df.head()

Client  Lat  Long Medicine  Dosage
0  Client 000   76    62     8334       2
1  Client 001   47   117     7f4f       2
2  Client 002    4   107     5519       1
3  Client 003  116    12     6f0c       1
4  Client 004   88    59     bbe0       2

In [5]:
# Generate Pharmacies
pharmacy_count = 10
pharmacy_names = [f'pharm{i}' for i in range(pharmacy_count)]

latitudes = np.random.randint(0, MAXLAT, pharmacy_count)
longitudes = np.random.randint(0, MAXLONG, pharmacy_count)

rx_df = pd.DataFrame({
    'Name': pharmacy_names,
    'Lat': latitudes,
    'Long': longitudes
})
rx_df

Name  Lat  Long
0  pharm0   92   162
1  pharm1  227   230
2  pharm2  135    67
3  pharm3  184   110
4  pharm4  190    67
5  pharm5  165   187
6  pharm6    9   224
7  pharm7  226    99
8  pharm8   47   105
9  pharm9   96   202

In [6]:
# Assign Customers to Pharmacies
# client are assigned to closest pharm

# distance between each client and each pharm
client_coords = cs_df[['Lat', 'Long']].values
pharm = np.column_stack((latitudes[:pharmacy_count], longitudes[:pharmacy_count]))

distances = np.sqrt(
    (
        (client_coords[:, np.newaxis, :] - pharm[np.newaxis, :, :]) ** 2
    ).sum(axis=2))


# Assign each customer to the closest pharmacy
closest_pharmacies = np.argmin(distances, axis=1)
cs_df['Assigned_Pharmacy'] = closest_pharmacies

cs_df.head()


Client  Lat  Long Medicine  Dosage  Assigned_Pharmacy
0  Client 000  181    15     6f0c       2                  4
1  Client 001   34   218     38b5       2                  6
2  Client 002   57   114     8334       2                  8
3  Client 003  178    77     7f4f       1                  4
4  Client 004  153   217     38b5       1                  5

In [21]:
total_cs_df = cs_df.groupby(['Assigned_Pharmacy','Medicine']).sum().reset_index().set_index('Assigned_Pharmacy')
total_cs_df

Medicine  Lat  Long  Dosage
Assigned_Pharmacy                            
0                     38b5  119   149       1
0                     5519   63   169       2
0                     6f0c   44   184       1
0                     7f4f   64   141       1
0                     8334  183   305       3
0                     ddfb  127   151       1
1                     3630  239   212       1
1                     5519  425   474       3
1                     6f0c  225   229       2
1                     7f4f  210   229       2
1                     8334  873   959       6
1                     bbe0  238   179       1
1                     ddfb  248   200       2
2                     246b  100    40       1
2                     3630  213   133       3
2                     5519  222    93       4
2                     7f4f  101    12       2
2                     eeed   95    23       2
3                     6f0c  291   241       3
3                     8334  205   125       2
3                     eeed  151   114       2
4                     246b  219    65       1
4                     3630  177     9       2
4                     6f0c  646    50       5
4                     7f4f  552   167       4
4                     eeed  194    28       2
5                     3630  356   347       3
5                     38b5  153   217       1
5                     5519  335   438       3
5                     6f0c  919   883       8
5                     8334  145   216       2
5                     ddfb  146   246       1
5                     eeed  154   193       2
6                     246b   28   229       1
6                     38b5   37   392       3
6                     5519   56   439       3
6                     bbe0    3   190       2
6                     eeed   30   197       1
7                     38b5  472   156       3
7                     5519  241   106       1
7                     7f4f  219   137       1
7                     bbe0  250   122       2
8                     246b  142   623       9
8                     3630   22    57       2
8                     38b5   94   146       4
8                     5519   14   265       3
8                     6f0c   45   140       2
8                     7f4f  111   235       3
8                     8334   57   114       2
8                     bbe0  110   225       4
8                     ddfb  157   465       5
8                     eeed   93   210       3
9                     246b   91   217       1
9                     38b5  216   461       4
9                     5519  195   440       3
9                     6f0c  221   413       4
9                     7f4f  100   204       2
9                     8334  307   703       5

In [22]:
pd.merge(rx_df, total_cs_df, right_index=True, left_index=True, how='left')

Name  Lat_x  Long_x  Demand Medicine  Lat_y  Long_y  Dosage
0  pharm0     92     162     NaN     38b5    119     149       1
0  pharm0     92     162     NaN     5519     63     169       2
0  pharm0     92     162     NaN     6f0c     44     184       1
0  pharm0     92     162     NaN     7f4f     64     141       1
0  pharm0     92     162     NaN     8334    183     305       3
0  pharm0     92     162     NaN     ddfb    127     151       1
1  pharm1    227     230     NaN     3630    239     212       1
1  pharm1    227     230     NaN     5519    425     474       3
1  pharm1    227     230     NaN     6f0c    225     229       2
1  pharm1    227     230     NaN     7f4f    210     229       2
1  pharm1    227     230     NaN     8334    873     959       6
1  pharm1    227     230     NaN     bbe0    238     179       1
1  pharm1    227     230     NaN     ddfb    248     200       2
2  pharm2    135      67     NaN     246b    100      40       1
2  pharm2    135      67     NaN     3630    213     133       3
2  pharm2    135      67     NaN     5519    222      93       4
2  pharm2    135      67     NaN     7f4f    101      12       2
2  pharm2    135      67     NaN     eeed     95      23       2
3  pharm3    184     110     NaN     6f0c    291     241       3
3  pharm3    184     110     NaN     8334    205     125       2
3  pharm3    184     110     NaN     eeed    151     114       2
4  pharm4    190      67     NaN     246b    219      65       1
4  pharm4    190      67     NaN     3630    177       9       2
4  pharm4    190      67     NaN     6f0c    646      50       5
4  pharm4    190      67     NaN     7f4f    552     167       4
4  pharm4    190      67     NaN     eeed    194      28       2
5  pharm5    165     187     NaN     3630    356     347       3
5  pharm5    165     187     NaN     38b5    153     217       1
5  pharm5    165     187     NaN     5519    335     438       3
5  pharm5    165     187     NaN     6f0c    919     883       8
5  pharm5    165     187     NaN     8334    145     216       2
5  pharm5    165     187     NaN     ddfb    146     246       1
5  pharm5    165     187     NaN     eeed    154     193       2
6  pharm6      9     224     NaN     246b     28     229       1
6  pharm6      9     224     NaN     38b5     37     392       3
6  pharm6      9     224     NaN     5519     56     439       3
6  pharm6      9     224     NaN     bbe0      3     190       2
6  pharm6      9     224     NaN     eeed     30     197       1
7  pharm7    226      99     NaN     38b5    472     156       3
7  pharm7    226      99     NaN     5519    241     106       1
7  pharm7    226      99     NaN     7f4f    219     137       1
7  pharm7    226      99     NaN     bbe0    250     122       2
8  pharm8     47     105     NaN     246b    142     623       9
8  pharm8     47     105     NaN     3630     22      57       2
8  pharm8     47     105     NaN     38b5     94     146       4
8  pharm8     47     105     NaN     5519     14     265       3
8  pharm8     47     105     NaN     6f0c     45     140       2
8  pharm8     47     105     NaN     7f4f    111     235       3
8  pharm8     47     105     NaN     8334     57     114       2
8  pharm8     47     105     NaN     bbe0    110     225       4
8  pharm8     47     105     NaN     ddfb    157     465       5
8  pharm8     47     105     NaN     eeed     93     210       3
9  pharm9     96     202     NaN     246b     91     217       1
9  pharm9     96     202     NaN     38b5    216     461       4
9  pharm9     96     202     NaN     5519    195     440       3
9  pharm9     96     202     NaN     6f0c    221     413       4
9  pharm9     96     202     NaN     7f4f    100     204       2
9  pharm9     96     202     NaN     8334    307     703       5

In [8]:
# Assign each pharmacy a list of medications that are required

# use assigned_pharmacy in cs_df to get the pharmacy name, and then get the medicine
# turn that into a counter and then into a dataframe into demand

for client in range(client_count):
    pharmacy = cs_df.loc[client, 'Assigned_Pharmacy']
    med = cs_df.loc[client, 'Medicine']
    dosage = cs_df.loc[client, 'Dosage']
    rx_df.loc[pharmacy, 'Demand'] = Counter({med: dosage})
    # needs to be fixed ^ TODO

rx_df 


ValueError: Incompatible indexer with Series